In [15]:
# basic imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import sys
from dotenv import load_dotenv

# load dotenv
load_dotenv()

%matplotlib inline

In [2]:
# authorize user and initialize spotipy instance
import spotipy
from spotipy.oauth2 import SpotifyOAuth

sp = spotipy.Spotify(auth_manager=SpotifyOAuth())

In [3]:
featured_playlists_sp = sp.featured_playlists(locale="EN")

In [4]:
for playlist in featured_playlists_sp['playlists']['items']:
    print(playlist['name'])

Calm Down
All Out 00s
Timeless Love Songs
Evening Acoustic
Dreamy Vibes
Brand New Chill
50 Great Female Voices
Bedroom Jams
Easy 90s
Chilled Dance
Relax & Unwind
Feelgood Pop


In [5]:
featured_playlists = [p for p in featured_playlists_sp['playlists']['items']]

In [6]:
tracks = []
for playlist in featured_playlists:
    p_items = sp.playlist_items(playlist['id'])
    print(playlist['name'] + ":", len(p_items['items']), "tracks")
    for element in p_items['items']:
        tracks.append(element)
print(len(tracks), "tracks collected")

Calm Down: 70 tracks
All Out 00s: 100 tracks
Timeless Love Songs: 100 tracks
Evening Acoustic: 100 tracks
Dreamy Vibes: 100 tracks
Brand New Chill: 100 tracks
50 Great Female Voices: 50 tracks
Bedroom Jams: 75 tracks
Easy 90s: 80 tracks
Chilled Dance: 46 tracks
Relax & Unwind: 100 tracks
Feelgood Pop: 50 tracks
971 tracks collected


In [7]:
tracks = [t['track'] for t in tracks]

In [8]:
def filter_track_meta(track):
    return {
        'name': track['name'],
        'artist': track['artists'][0]['name']
    }

In [9]:
def get_audio_features(track):
    audio_features = sp.audio_features(track['id'])[0]
    audio_features.pop('time_signature')
    audio_features.pop('uri')
    audio_features.pop('track_href')
    audio_features.pop('type')
    audio_features.pop('analysis_url')
    audio_features.pop('duration_ms')
    return audio_features

In [10]:
def blend_track_data(track):
    audio_features = get_audio_features(track)
    track_meta = filter_track_meta(track)
    track = track_meta
    for key in audio_features.keys():
        track[key] = audio_features[key]
    return track

In [11]:
blend_track_data(tracks[0])

{'name': 'Mystery of Love (From the Original Motion Picture “Call Me by Your Name”)',
 'artist': 'Sufjan Stevens',
 'danceability': 0.365,
 'energy': 0.273,
 'key': 4,
 'loudness': -16.526,
 'mode': 0,
 'speechiness': 0.038,
 'acousticness': 0.94,
 'instrumentalness': 0.431,
 'liveness': 0.109,
 'valence': 0.238,
 'tempo': 132.285,
 'id': '0oTtnnedK0C4unALxVTPhz'}

In [20]:
tracks_with_attributes = []
count = 0
tracks_len = len(tracks)
for track in tracks:
    time.sleep(0.05)
    count += 1
    print("\r", "#" + str(count), f"{(count/tracks_len)*100:5.2f}%", end="")
    tracks_with_attributes.append(blend_track_data(track))

 #971 100.00%

In [35]:
tracks_with_attributes[97]

{'name': 'I Knew You Were Trouble',
 'artist': 'Taylor Swift',
 'danceability': 0.499,
 'energy': 0.467,
 'key': 6,
 'loudness': -6.744,
 'mode': 1,
 'speechiness': 0.0461,
 'acousticness': 0.00593,
 'instrumentalness': 1.65e-06,
 'liveness': 0.0361,
 'valence': 0.637,
 'tempo': 76.76,
 'id': '6RjKVXi31fui9m4dbiGRrL'}

In [23]:
import pickle

In [24]:
with open("./data/spotify_featured_songs.pkl", "wb") as f:
    pickle.dump(tracks_with_attributes, f)